# Notebook For NLP (Natural Language Processing).
<hr>

* This Notebook is used for the process of social listening.
* This notebook processes text (unstructured data)
* Initially data brought from SentiOne


# NPL Process

In [1]:
import sys
import os

# Get the parent directory of 'topology/'
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)  # Add 'parent/' to Python's path

from config import settings, paths

In [2]:
import config.m365.auth.microsoft_graph as mg

In [3]:
from config import settings, paths
from NLP_optimized import *

El directorio ya existe: C:/Users/Santi/Documents/atinna/repos/escuchas\Azul\salud\2025-04-01_2025-04-30\v1.0


## User Parameter

In [4]:
# Autenticación
token_response = mg.authenticate(client_id= settings.id_client, auth= settings.AUTHORITY, scopes= settings.GRAPH_SCOPES)
headers = mg.get_headers(token_response)

✅ Usando sesión existente para santiago.munera@atinna.co


In [5]:
f"{settings.dir_cliente}/{settings.dir_escucha}/{settings.dir_periodo}/{settings.dir_version}"

'Azul/salud/2025-04-01_2025-04-30/v1.0'

In [6]:
team_id = mg.get_team_id(headers, settings.dir_cliente) if headers else None
sharepoint_site = mg.get_sharepoint_site(headers, team_id)
site_id = sharepoint_site["id"]
drive_items = mg.get_drive_items(headers, site_id)
dev_folder = next((file for file in drive_items if file["name"].lower() == "desarrollo"), None)
dev_folder_id = dev_folder["id"]
dev_files = mg.get_folder_contents(headers, site_id, dev_folder_id)

In [7]:
os.makedirs(paths.paths['local']['output']['lda']['base'], exist_ok=True)
os.makedirs(paths.paths['local']['output']['lda']['models'], exist_ok=True)
os.makedirs(paths.paths['local']['output']['lda']['figures'], exist_ok=True)

In [8]:
mg.create_folder(headers, site_id, dev_folder_id, paths.paths['cloud']['output']['lda']['base'], separator="/")
id_folder = mg.get_folder_id(headers, site_id, dev_folder_id, paths.paths['cloud']['output']['lda']['base'], separator="/")

📁 Intentando crear 'salud'...
📂 La carpeta 'salud' ya existe.
📁 Intentando crear '2025-04-01_2025-04-30'...
📂 La carpeta '2025-04-01_2025-04-30' ya existe.
📁 Intentando crear 'v1.0'...
📂 La carpeta 'v1.0' ya existe.
📁 Intentando crear 'output'...
📂 La carpeta 'output' ya existe.
📁 Intentando crear 'lda'...
📂 La carpeta 'lda' ya existe.


In [9]:
"""id_folder = mg.get_folder_id(headers, site_id, dev_folder_id, paths.paths['cloud']['data']['base'], separator="/")
files = mg.get_folder_contents(headers, site_id, id_folder)
print("\n📂 Archivos en la carpeta Desarrollo:")
for file in files:
    print(f"- {file['name']} ({file['id']})")"""

'id_folder = mg.get_folder_id(headers, site_id, dev_folder_id, paths.paths[\'cloud\'][\'data\'][\'base\'], separator="/")\nfiles = mg.get_folder_contents(headers, site_id, id_folder)\nprint("\n📂 Archivos en la carpeta Desarrollo:")\nfor file in files:\n    print(f"- {file[\'name\']} ({file[\'id\']})")'

In [10]:
file_name = "espectro_politico-2025-04-01_2025-04-30.csv"
#file_metadata = next((file for file in files if file["name"] == file_name), None)
#file_id = file_metadata["id"]
local_path = paths.paths['local']['data']['base']
file_path = os.path.join(local_path, file_name)

if not os.path.exists(file_path):
    print(f"File not found: {local_path}. Downloading...")
    #file_metadata = next((file for file in files if file["name"] == file_name), None)
    #file_id = file_metadata["id"]

    #file = mg.download_file(headers, site_id, file_id, local_path, create_missing_dirs=True)
    print(f"Download complete: {local_path}")
    
else:
    print(f"File already exists: {local_path}")

File not found: C:/Users/Santi/Documents/atinna/repos/escuchas\Azul\salud\2025-04-01_2025-04-30\v1.0\data. Downloading...
Download complete: C:/Users/Santi/Documents/atinna/repos/escuchas\Azul\salud\2025-04-01_2025-04-30\v1.0\data


In [11]:
lib='polars'#polars, pandas
optimization=True

#column names and column for use.
names = {
'Autor':'author',
'Contenido de la publicación':'content',#Minimum required
'Creado':'date',
'Contexto':'context',
'Link para la fuente':'link',
'Grupo de dominio':'domain_group',#Minimum required
'País':'country',
'Tipo específico':'type'
}

# Filter of values for columns
filter_list = ['Twitter','Facebook','Instagram','Video','TikTok','Reddit']

new_stop = [
    'solo', 'ahora', 'usted', 'bueno', 'si', 'siempre', 'hoy', 'día', 'dias', 'favor', 'ver', 'verdad', 
    'verdadero', 'verdadera', 'así', 'decir', 'pasar', 'deber', 'ser', 'nuevo', 'nueva', 'nuevos', 
    'nuevas', 'hacer', 'paí', 'ud', 'poder', 'querer', 'ir', 'dar', 'saber', 'venir', 'tener', 'creer', 
    'tanto', 'bien', 'claro', 'según', 'sino', 'mismo', 'q', 'señor', 'hecho', 'hablar', 'llevar', 
    'seguir', 'además', 'entonces', 'esperar', 'mundo', 'llegar', 'parecer', 'nunca', 'salir', 'persona', 
    'dejar', 'cómo', 'año', 'entonce', 'ma', 'pue', 'dío', 'tal', 'volver', 'ahí', 'aquí', 'meno', 'dos', 
    'nar', 'gracia', 'luego', 'único', 'vez', 'dar él', 'dar', 'él', 'ella', 'ello', 'mientras', 'caso', 
    'cada', 'dio', 'mejor', 'pueblo', 'gente', 'mas', 'menos', 'gracias', 'años', 'más', 'cosa', 'momento', 
    'asi', 'ano', 'mes', 'semana', 'semanas', 'aca', 'peor', 'meses', 'sera', 'mil', 'aqui', 'aun', 'sr', 
    'ahi', 'don', 'dr', 'sra', 'srita', 'habia', 'casi', 'unico', 'ala', 'usar', 'llego', 'porq', 'xk', 
    'xq', 'pq', 'pork', 'pues', 'Argentina', 'che', 'vos', 'boludo', 'mina', 'pibe', 'jajaja', 'jajajaja', 'jaja',
    'haha', 'hahaha', 'hahahaha'
]

sample_frac=None#0.25

n_volume=None#10000

optimization = True

min_topic = 4#2

max_topic = 10#2

k=None

sep=','

## Processing

In [12]:
%%time


k_recommended = Social_listening_data_process_and_modeling_LDA_OPTIMIZED(
        sep=sep,
        path_data=file_path,
        path_output= paths.paths['local']['output']['lda']['base'],
        path_output_model= paths.paths['local']['output']['lda']['models'],
        column_names=names,
        filter_list=filter_list,
        new_stop=new_stop,
        sample_frac=sample_frac,
        n_volume=n_volume,
        min_topics=min_topic,
        max_topics=max_topic,
        step_size=1,
        k=k,
        n_cpu=6,
        n_cpu_lda=8,
)

print(f"✅ Procesamiento completado! K recomendado: {k_recommended}")

ERROR: 🚀 Usando optimización ULTRA automáticamente para máximo rendimiento


NameError: name 'Social_listening_data_process_and_modeling_LDA_ULTRA_OPTIMIZED' is not defined

In [13]:
%%time

pipeline_result = {'success': False, 'error': 'Pipeline no ejecutado'} # Inicializar resultado
memory_manager_instance = MemoryManager() # Instanciar para uso en esta celda

print("🚀 INICIANDO PIPELINE LDA OPTIMIZADO (CON run_lda_pipeline_wrapper)")
print("=" * 60)
start_datetime_pipeline = datetime.now()
print(f"📅 Inicio: {start_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Memoria inicial (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print()

if not Path(file_path).exists():
    logger.error(f"El archivo de datos '{file_path}' no existe. No se puede ejecutar el pipeline.")
    pipeline_result = {'success': False, 'error': f'Archivo de datos no encontrado: {file_path}'}
else:
    try:
        # Usar la función wrapper del script nlp_lda_optimized.py
        pipeline_result = run_lda_pipeline_wrapper(
            path_data=str(file_path), # Asegurar que es string
            path_output=str(local_output_lda_base), # Ruta base para todas las salidas del pipeline
            sep=config_pipeline['sep'],
            column_mapping=config_pipeline['column_mapping'],
            filter_list=config_pipeline['filter_list'],
            new_stop=config_pipeline['new_stop'],
            min_topics=config_pipeline['min_topics'],
            max_topics=config_pipeline['max_topics'],
            step_size=config_pipeline['step_size'],
            k=config_pipeline['k'],
            n_cpu_lda=config_pipeline['n_cpu_lda'],
            sample_frac=config_pipeline['sample_frac'],
            n_volume=config_pipeline['n_volume'],
            content_col_name=config_pipeline['content_col_name'],
            domain_col_name=config_pipeline['domain_col_name'],
            date_col_name=config_pipeline['date_col_name']
        )
        
        if pipeline_result.get('success', False):
            print("\n✅ PIPELINE COMPLETADO EXITOSAMENTE")
            print("=" * 40)
            print(f"🎯 K recomendado: {pipeline_result.get('k_recommended', 'No determinado')}")
            print(f"📁 Resultados guardados en la base: {pipeline_result.get('path_output_base')}")
            print(f"📄 Datos procesados en: {pipeline_result.get('path_processed_data')}")
            print(f"🤖 Modelos LDA en: {pipeline_result.get('path_lda_models')}")
            print(f"📈 Resultados de coherencia LDA en: {pipeline_result.get('path_lda_results')}")
        else:
            print("\n❌ ERROR DURANTE LA EJECUCIÓN DEL PIPELINE")
            print("=" * 40)
            print(f"Tipo de Error: {pipeline_result.get('error_type', 'Desconocido')}")
            print(f"Mensaje: {pipeline_result.get('error_message', 'No hay mensaje detallado.')}")
            logger.error(f"Pipeline falló: {pipeline_result.get('error_type', '')} - {pipeline_result.get('error_message', '')}")
            
    except Exception as e_pipeline_cell:
        print(f"\n💥 EXCEPCIÓN NO CONTROLADA EN LA CELDA DEL PIPELINE: {e_pipeline_cell}")
        logger.error(f"Error crítico en la celda de ejecución del pipeline: {e_pipeline_cell}", exc_info=True)
        pipeline_result = {'success': False, 'error_type': type(e_pipeline_cell).__name__, 'error_message': str(e_pipeline_cell)}

end_datetime_pipeline = datetime.now()
print(f"\n💾 Memoria final (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print(f"📅 Fin: {end_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duración total de la celda: {end_datetime_pipeline - start_datetime_pipeline}")

NameError: name 'MemoryManager' is not defined

In [14]:
%%time

pipeline_result = {'success': False, 'error': 'Pipeline no ejecutado'} # Inicializar resultado
memory_manager_instance = MemoryManager() # Instanciar para uso en esta celda

print("🚀 INICIANDO PIPELINE LDA OPTIMIZADO (CON run_lda_pipeline_wrapper)")
print("=" * 60)
start_datetime_pipeline = datetime.now()
print(f"📅 Inicio: {start_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Memoria inicial (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print()

if not Path(file_path).exists():
    logger.error(f"El archivo de datos '{file_path}' no existe. No se puede ejecutar el pipeline.")
    pipeline_result = {'success': False, 'error': f'Archivo de datos no encontrado: {file_path}'}
else:
    try:
        # Usar la función wrapper del script nlp_lda_optimized.py
        pipeline_result = run_lda_pipeline_wrapper(
            path_data=str(file_path), # Asegurar que es string
            path_output=str(local_output_lda_base), # Ruta base para todas las salidas del pipeline
            sep=config_pipeline['sep'],
            column_mapping=config_pipeline['column_mapping'],
            filter_list=config_pipeline['filter_list'],
            new_stop=config_pipeline['new_stop'],
            min_topics=config_pipeline['min_topics'],
            max_topics=config_pipeline['max_topics'],
            step_size=config_pipeline['step_size'],
            k=config_pipeline['k'],
            n_cpu_lda=config_pipeline['n_cpu_lda'],
            sample_frac=config_pipeline['sample_frac'],
            n_volume=config_pipeline['n_volume'],
            content_col_name=config_pipeline['content_col_name'],
            domain_col_name=config_pipeline['domain_col_name'],
            date_col_name=config_pipeline['date_col_name']
        )
        
        if pipeline_result.get('success', False):
            print("\n✅ PIPELINE COMPLETADO EXITOSAMENTE")
            print("=" * 40)
            print(f"🎯 K recomendado: {pipeline_result.get('k_recommended', 'No determinado')}")
            print(f"📁 Resultados guardados en la base: {pipeline_result.get('path_output_base')}")
            print(f"📄 Datos procesados en: {pipeline_result.get('path_processed_data')}")
            print(f"🤖 Modelos LDA en: {pipeline_result.get('path_lda_models')}")
            print(f"📈 Resultados de coherencia LDA en: {pipeline_result.get('path_lda_results')}")
        else:
            print("\n❌ ERROR DURANTE LA EJECUCIÓN DEL PIPELINE")
            print("=" * 40)
            print(f"Tipo de Error: {pipeline_result.get('error_type', 'Desconocido')}")
            print(f"Mensaje: {pipeline_result.get('error_message', 'No hay mensaje detallado.')}")
            logger.error(f"Pipeline falló: {pipeline_result.get('error_type', '')} - {pipeline_result.get('error_message', '')}")
            
    except Exception as e_pipeline_cell:
        print(f"\n💥 EXCEPCIÓN NO CONTROLADA EN LA CELDA DEL PIPELINE: {e_pipeline_cell}")
        logger.error(f"Error crítico en la celda de ejecución del pipeline: {e_pipeline_cell}", exc_info=True)
        pipeline_result = {'success': False, 'error_type': type(e_pipeline_cell).__name__, 'error_message': str(e_pipeline_cell)}

end_datetime_pipeline = datetime.now()
print(f"\n💾 Memoria final (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print(f"📅 Fin: {end_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duración total de la celda: {end_datetime_pipeline - start_datetime_pipeline}")

NameError: name 'MemoryManager' is not defined

In [15]:
%%time

pipeline_result = {'success': False, 'error': 'Pipeline no ejecutado'} # Inicializar resultado
memory_manager_instance = MemoryManager() # Instanciar para uso en esta celda

print("🚀 INICIANDO PIPELINE LDA OPTIMIZADO (CON run_lda_pipeline_wrapper)")
print("=" * 60)
start_datetime_pipeline = datetime.now()
print(f"📅 Inicio: {start_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Memoria inicial (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print()

if not Path(file_path).exists():
    logger.error(f"El archivo de datos '{file_path}' no existe. No se puede ejecutar el pipeline.")
    pipeline_result = {'success': False, 'error': f'Archivo de datos no encontrado: {file_path}'}
else:
    try:
        # Usar la función wrapper del script nlp_lda_optimized.py
        pipeline_result = run_lda_pipeline_wrapper(
            path_data=str(file_path), # Asegurar que es string
            path_output=str(local_output_lda_base), # Ruta base para todas las salidas del pipeline
            sep=config_pipeline['sep'],
            column_mapping=config_pipeline['column_mapping'],
            filter_list=config_pipeline['filter_list'],
            new_stop=config_pipeline['new_stop'],
            min_topics=config_pipeline['min_topics'],
            max_topics=config_pipeline['max_topics'],
            step_size=config_pipeline['step_size'],
            k=config_pipeline['k'],
            n_cpu_lda=config_pipeline['n_cpu_lda'],
            sample_frac=config_pipeline['sample_frac'],
            n_volume=config_pipeline['n_volume'],
            content_col_name=config_pipeline['content_col_name'],
            domain_col_name=config_pipeline['domain_col_name'],
            date_col_name=config_pipeline['date_col_name']
        )
        
        if pipeline_result.get('success', False):
            print("\n✅ PIPELINE COMPLETADO EXITOSAMENTE")
            print("=" * 40)
            print(f"🎯 K recomendado: {pipeline_result.get('k_recommended', 'No determinado')}")
            print(f"📁 Resultados guardados en la base: {pipeline_result.get('path_output_base')}")
            print(f"📄 Datos procesados en: {pipeline_result.get('path_processed_data')}")
            print(f"🤖 Modelos LDA en: {pipeline_result.get('path_lda_models')}")
            print(f"📈 Resultados de coherencia LDA en: {pipeline_result.get('path_lda_results')}")
        else:
            print("\n❌ ERROR DURANTE LA EJECUCIÓN DEL PIPELINE")
            print("=" * 40)
            print(f"Tipo de Error: {pipeline_result.get('error_type', 'Desconocido')}")
            print(f"Mensaje: {pipeline_result.get('error_message', 'No hay mensaje detallado.')}")
            logger.error(f"Pipeline falló: {pipeline_result.get('error_type', '')} - {pipeline_result.get('error_message', '')}")
            
    except Exception as e_pipeline_cell:
        print(f"\n💥 EXCEPCIÓN NO CONTROLADA EN LA CELDA DEL PIPELINE: {e_pipeline_cell}")
        logger.error(f"Error crítico en la celda de ejecución del pipeline: {e_pipeline_cell}", exc_info=True)
        pipeline_result = {'success': False, 'error_type': type(e_pipeline_cell).__name__, 'error_message': str(e_pipeline_cell)}

end_datetime_pipeline = datetime.now()
print(f"\n💾 Memoria final (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print(f"📅 Fin: {end_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duración total de la celda: {end_datetime_pipeline - start_datetime_pipeline}")

NameError: name 'MemoryManager' is not defined

In [16]:
%%time

pipeline_result = {'success': False, 'error': 'Pipeline no ejecutado'} # Inicializar resultado
memory_manager_instance = MemoryManager() # Instanciar para uso en esta celda

print("🚀 INICIANDO PIPELINE LDA OPTIMIZADO (CON run_lda_pipeline_wrapper)")
print("=" * 60)
start_datetime_pipeline = datetime.now()
print(f"📅 Inicio: {start_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Memoria inicial (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print()

if not Path(file_path).exists():
    logger.error(f"El archivo de datos '{file_path}' no existe. No se puede ejecutar el pipeline.")
    pipeline_result = {'success': False, 'error': f'Archivo de datos no encontrado: {file_path}'}
else:
    try:
        # Usar la función wrapper del script nlp_lda_optimized.py
        pipeline_result = run_lda_pipeline_wrapper(
            path_data=str(file_path), # Asegurar que es string
            path_output=str(local_output_lda_base), # Ruta base para todas las salidas del pipeline
            sep=config_pipeline['sep'],
            column_mapping=config_pipeline['column_mapping'],
            filter_list=config_pipeline['filter_list'],
            new_stop=config_pipeline['new_stop'],
            min_topics=config_pipeline['min_topics'],
            max_topics=config_pipeline['max_topics'],
            step_size=config_pipeline['step_size'],
            k=config_pipeline['k'],
            n_cpu_lda=config_pipeline['n_cpu_lda'],
            sample_frac=config_pipeline['sample_frac'],
            n_volume=config_pipeline['n_volume'],
            content_col_name=config_pipeline['content_col_name'],
            domain_col_name=config_pipeline['domain_col_name'],
            date_col_name=config_pipeline['date_col_name']
        )
        
        if pipeline_result.get('success', False):
            print("\n✅ PIPELINE COMPLETADO EXITOSAMENTE")
            print("=" * 40)
            print(f"🎯 K recomendado: {pipeline_result.get('k_recommended', 'No determinado')}")
            print(f"📁 Resultados guardados en la base: {pipeline_result.get('path_output_base')}")
            print(f"📄 Datos procesados en: {pipeline_result.get('path_processed_data')}")
            print(f"🤖 Modelos LDA en: {pipeline_result.get('path_lda_models')}")
            print(f"📈 Resultados de coherencia LDA en: {pipeline_result.get('path_lda_results')}")
        else:
            print("\n❌ ERROR DURANTE LA EJECUCIÓN DEL PIPELINE")
            print("=" * 40)
            print(f"Tipo de Error: {pipeline_result.get('error_type', 'Desconocido')}")
            print(f"Mensaje: {pipeline_result.get('error_message', 'No hay mensaje detallado.')}")
            logger.error(f"Pipeline falló: {pipeline_result.get('error_type', '')} - {pipeline_result.get('error_message', '')}")
            
    except Exception as e_pipeline_cell:
        print(f"\n💥 EXCEPCIÓN NO CONTROLADA EN LA CELDA DEL PIPELINE: {e_pipeline_cell}")
        logger.error(f"Error crítico en la celda de ejecución del pipeline: {e_pipeline_cell}", exc_info=True)
        pipeline_result = {'success': False, 'error_type': type(e_pipeline_cell).__name__, 'error_message': str(e_pipeline_cell)}

end_datetime_pipeline = datetime.now()
print(f"\n💾 Memoria final (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print(f"📅 Fin: {end_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duración total de la celda: {end_datetime_pipeline - start_datetime_pipeline}")

NameError: name 'MemoryManager' is not defined

In [17]:
%%time

pipeline_result = {'success': False, 'error': 'Pipeline no ejecutado'} # Inicializar resultado
memory_manager_instance = MemoryManager() # Instanciar para uso en esta celda

print("🚀 INICIANDO PIPELINE LDA OPTIMIZADO (CON run_lda_pipeline_wrapper)")
print("=" * 60)
start_datetime_pipeline = datetime.now()
print(f"📅 Inicio: {start_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Memoria inicial (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print()

if not Path(file_path).exists():
    logger.error(f"El archivo de datos '{file_path}' no existe. No se puede ejecutar el pipeline.")
    pipeline_result = {'success': False, 'error': f'Archivo de datos no encontrado: {file_path}'}
else:
    try:
        # Usar la función wrapper del script nlp_lda_optimized.py
        pipeline_result = run_lda_pipeline_wrapper(
            path_data=str(file_path), # Asegurar que es string
            path_output=str(local_output_lda_base), # Ruta base para todas las salidas del pipeline
            sep=config_pipeline['sep'],
            column_mapping=config_pipeline['column_mapping'],
            filter_list=config_pipeline['filter_list'],
            new_stop=config_pipeline['new_stop'],
            min_topics=config_pipeline['min_topics'],
            max_topics=config_pipeline['max_topics'],
            step_size=config_pipeline['step_size'],
            k=config_pipeline['k'],
            n_cpu_lda=config_pipeline['n_cpu_lda'],
            sample_frac=config_pipeline['sample_frac'],
            n_volume=config_pipeline['n_volume'],
            content_col_name=config_pipeline['content_col_name'],
            domain_col_name=config_pipeline['domain_col_name'],
            date_col_name=config_pipeline['date_col_name']
        )
        
        if pipeline_result.get('success', False):
            print("\n✅ PIPELINE COMPLETADO EXITOSAMENTE")
            print("=" * 40)
            print(f"🎯 K recomendado: {pipeline_result.get('k_recommended', 'No determinado')}")
            print(f"📁 Resultados guardados en la base: {pipeline_result.get('path_output_base')}")
            print(f"📄 Datos procesados en: {pipeline_result.get('path_processed_data')}")
            print(f"🤖 Modelos LDA en: {pipeline_result.get('path_lda_models')}")
            print(f"📈 Resultados de coherencia LDA en: {pipeline_result.get('path_lda_results')}")
        else:
            print("\n❌ ERROR DURANTE LA EJECUCIÓN DEL PIPELINE")
            print("=" * 40)
            print(f"Tipo de Error: {pipeline_result.get('error_type', 'Desconocido')}")
            print(f"Mensaje: {pipeline_result.get('error_message', 'No hay mensaje detallado.')}")
            logger.error(f"Pipeline falló: {pipeline_result.get('error_type', '')} - {pipeline_result.get('error_message', '')}")
            
    except Exception as e_pipeline_cell:
        print(f"\n💥 EXCEPCIÓN NO CONTROLADA EN LA CELDA DEL PIPELINE: {e_pipeline_cell}")
        logger.error(f"Error crítico en la celda de ejecución del pipeline: {e_pipeline_cell}", exc_info=True)
        pipeline_result = {'success': False, 'error_type': type(e_pipeline_cell).__name__, 'error_message': str(e_pipeline_cell)}

end_datetime_pipeline = datetime.now()
print(f"\n💾 Memoria final (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print(f"📅 Fin: {end_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duración total de la celda: {end_datetime_pipeline - start_datetime_pipeline}")

NameError: name 'MemoryManager' is not defined

In [18]:
%%time

pipeline_result = {'success': False, 'error': 'Pipeline no ejecutado'} # Inicializar resultado
memory_manager_instance = MemoryManager() # Instanciar para uso en esta celda

print("🚀 INICIANDO PIPELINE LDA OPTIMIZADO (CON run_lda_pipeline_wrapper)")
print("=" * 60)
start_datetime_pipeline = datetime.now()
print(f"📅 Inicio: {start_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Memoria inicial (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print()

if not Path(file_path).exists():
    logger.error(f"El archivo de datos '{file_path}' no existe. No se puede ejecutar el pipeline.")
    pipeline_result = {'success': False, 'error': f'Archivo de datos no encontrado: {file_path}'}
else:
    try:
        # Usar la función wrapper del script nlp_lda_optimized.py
        pipeline_result = run_lda_pipeline_wrapper(
            path_data=str(file_path), # Asegurar que es string
            path_output=str(local_output_lda_base), # Ruta base para todas las salidas del pipeline
            sep=config_pipeline['sep'],
            column_mapping=config_pipeline['column_mapping'],
            filter_list=config_pipeline['filter_list'],
            new_stop=config_pipeline['new_stop'],
            min_topics=config_pipeline['min_topics'],
            max_topics=config_pipeline['max_topics'],
            step_size=config_pipeline['step_size'],
            k=config_pipeline['k'],
            n_cpu_lda=config_pipeline['n_cpu_lda'],
            sample_frac=config_pipeline['sample_frac'],
            n_volume=config_pipeline['n_volume'],
            content_col_name=config_pipeline['content_col_name'],
            domain_col_name=config_pipeline['domain_col_name'],
            date_col_name=config_pipeline['date_col_name']
        )
        
        if pipeline_result.get('success', False):
            print("\n✅ PIPELINE COMPLETADO EXITOSAMENTE")
            print("=" * 40)
            print(f"🎯 K recomendado: {pipeline_result.get('k_recommended', 'No determinado')}")
            print(f"📁 Resultados guardados en la base: {pipeline_result.get('path_output_base')}")
            print(f"📄 Datos procesados en: {pipeline_result.get('path_processed_data')}")
            print(f"🤖 Modelos LDA en: {pipeline_result.get('path_lda_models')}")
            print(f"📈 Resultados de coherencia LDA en: {pipeline_result.get('path_lda_results')}")
        else:
            print("\n❌ ERROR DURANTE LA EJECUCIÓN DEL PIPELINE")
            print("=" * 40)
            print(f"Tipo de Error: {pipeline_result.get('error_type', 'Desconocido')}")
            print(f"Mensaje: {pipeline_result.get('error_message', 'No hay mensaje detallado.')}")
            logger.error(f"Pipeline falló: {pipeline_result.get('error_type', '')} - {pipeline_result.get('error_message', '')}")
            
    except Exception as e_pipeline_cell:
        print(f"\n💥 EXCEPCIÓN NO CONTROLADA EN LA CELDA DEL PIPELINE: {e_pipeline_cell}")
        logger.error(f"Error crítico en la celda de ejecución del pipeline: {e_pipeline_cell}", exc_info=True)
        pipeline_result = {'success': False, 'error_type': type(e_pipeline_cell).__name__, 'error_message': str(e_pipeline_cell)}

end_datetime_pipeline = datetime.now()
print(f"\n💾 Memoria final (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print(f"📅 Fin: {end_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duración total de la celda: {end_datetime_pipeline - start_datetime_pipeline}")

NameError: name 'MemoryManager' is not defined

In [19]:
%%time

pipeline_result = {'success': False, 'error': 'Pipeline no ejecutado'} # Inicializar resultado
memory_manager_instance = MemoryManager() # Instanciar para uso en esta celda

print("🚀 INICIANDO PIPELINE LDA OPTIMIZADO (CON run_lda_pipeline_wrapper)")
print("=" * 60)
start_datetime_pipeline = datetime.now()
print(f"📅 Inicio: {start_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Memoria inicial (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print()

if not Path(file_path).exists():
    logger.error(f"El archivo de datos '{file_path}' no existe. No se puede ejecutar el pipeline.")
    pipeline_result = {'success': False, 'error': f'Archivo de datos no encontrado: {file_path}'}
else:
    try:
        # Usar la función wrapper del script nlp_lda_optimized.py
        pipeline_result = run_lda_pipeline_wrapper(
            path_data=str(file_path), # Asegurar que es string
            path_output=str(local_output_lda_base), # Ruta base para todas las salidas del pipeline
            sep=config_pipeline['sep'],
            column_mapping=config_pipeline['column_mapping'],
            filter_list=config_pipeline['filter_list'],
            new_stop=config_pipeline['new_stop'],
            min_topics=config_pipeline['min_topics'],
            max_topics=config_pipeline['max_topics'],
            step_size=config_pipeline['step_size'],
            k=config_pipeline['k'],
            n_cpu_lda=config_pipeline['n_cpu_lda'],
            sample_frac=config_pipeline['sample_frac'],
            n_volume=config_pipeline['n_volume'],
            content_col_name=config_pipeline['content_col_name'],
            domain_col_name=config_pipeline['domain_col_name'],
            date_col_name=config_pipeline['date_col_name']
        )
        
        if pipeline_result.get('success', False):
            print("\n✅ PIPELINE COMPLETADO EXITOSAMENTE")
            print("=" * 40)
            print(f"🎯 K recomendado: {pipeline_result.get('k_recommended', 'No determinado')}")
            print(f"📁 Resultados guardados en la base: {pipeline_result.get('path_output_base')}")
            print(f"📄 Datos procesados en: {pipeline_result.get('path_processed_data')}")
            print(f"🤖 Modelos LDA en: {pipeline_result.get('path_lda_models')}")
            print(f"📈 Resultados de coherencia LDA en: {pipeline_result.get('path_lda_results')}")
        else:
            print("\n❌ ERROR DURANTE LA EJECUCIÓN DEL PIPELINE")
            print("=" * 40)
            print(f"Tipo de Error: {pipeline_result.get('error_type', 'Desconocido')}")
            print(f"Mensaje: {pipeline_result.get('error_message', 'No hay mensaje detallado.')}")
            logger.error(f"Pipeline falló: {pipeline_result.get('error_type', '')} - {pipeline_result.get('error_message', '')}")
            
    except Exception as e_pipeline_cell:
        print(f"\n💥 EXCEPCIÓN NO CONTROLADA EN LA CELDA DEL PIPELINE: {e_pipeline_cell}")
        logger.error(f"Error crítico en la celda de ejecución del pipeline: {e_pipeline_cell}", exc_info=True)
        pipeline_result = {'success': False, 'error_type': type(e_pipeline_cell).__name__, 'error_message': str(e_pipeline_cell)}

end_datetime_pipeline = datetime.now()
print(f"\n💾 Memoria final (notebook): {memory_manager_instance.get_memory_usage():.2f} MB")
print(f"📅 Fin: {end_datetime_pipeline.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️ Duración total de la celda: {end_datetime_pipeline - start_datetime_pipeline}")

NameError: name 'MemoryManager' is not defined

# Outputs

## Metadata
Contains information about the process.

In [20]:
path_metadata = os.path.join(paths.paths['local']['output']['lda']['base'],"metadata.json")
metadata = json_read(path_metadata)
metadata

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Santi/Documents/atinna/repos/escuchas\\Azul\\salud\\2025-04-01_2025-04-30\\v1.0\\lda\\metadata.json'

## Hashtags frequency.

In [ ]:
[i for i in os.listdir(paths.paths['local']['output']['lda']['base']) if 'Hash'in i]

['Hash_global_polars.csv']

In [ ]:
hash= [local_reading(path=paths.paths['local']['output']['lda']['base']+i) for i in os.listdir(paths.paths['local']['output']['lda']['base']) if 'Hash'in i]
hash[0]

## Most repeated posts.

In [ ]:
# Repe= [local_reading(path=paths['local']['output']['base']+i) for i in os.listdir(paths['local']['output']['base']) if 'Repea'in i]
# df_repe = Repe[0].head(15).set_index('content')
# df_repe [df_repe['count']>1]

## Network Distribution.

In [ ]:
Netw= [local_reading(path=paths.paths['local']['output']['lda']['base']+i) for i in os.listdir(paths.paths['local']['output']['lda']['base']) if 'net'in i]
Netw

[None]

# Sync

In [ ]:
mg.upload_folder(headers, site_id, dev_folder_id, paths.paths['local']['output']['lda']['base'], paths.paths['cloud']['output']['lda']['base'], separator="/", conflict_behavior="fail", skip_on_error=False)

📂 Carpeta 'espectro_politico/2025-04-01_2025-04-30/v3.0/output/lda' encontrada o creada en SharePoint con ID: 01R4PBSE4OG4XZ5T4TRVFJNG5STBW2AFHK
📂 Carpeta 'espectro_politico/2025-04-01_2025-04-30/v3.0/output/lda/figures' encontrada o creada en SharePoint con ID: 01R4PBSEZI4PHVD6FUQFCZBVEZOGJEUSP3
❌ Error uploading file 'Hash_global_polars.csv': The specified item name already exists.
❌ Error al subir el archivo 'Hash_global_polars.csv'.


{'success': False,
 'files_uploaded': 0,
 'files_failed': 1,
 'folders_created': 0,
 'errors': ["Error al subir el archivo 'Hash_global_polars.csv'."]}